<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Validation/ResNet50_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/Proper_dataset/train"
test_dir = "/content/drive/MyDrive/Proper_dataset/val"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), class_mode='binary')

Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load ResNet50 without top layers
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)  # Binary classification

model_resnet = Model(inputs=base_model.input, outputs=output)

# Compile model
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
# Train model
model_resnet.fit(train_data, validation_data=test_data, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8008 - loss: 0.6002

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 592s 7s/step - accuracy: 0.8012 - loss: 0.5987 - val_accuracy: 0.8479 - val_loss: 0.3667
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 454s 5s/step - accuracy: 0.8505 - loss: 0.3894 - val_accuracy: 0.8479 - val_loss: 0.3695
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 424s 5s/step - accuracy: 0.8525 - loss: 0.3584 - val_accuracy: 0.8479 - val_loss: 0.3538
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 450s 5s/step - accuracy: 0.8573 - loss: 0.3567 - val_accuracy: 0.8479 - val_loss: 0.3155
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 422s 5s/step - accuracy: 0.8538 - loss: 0.3385 - val_accuracy: 0.8633 - val_loss: 0.3334
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 426s 5s/step - accuracy: 0.8545 - loss: 0.3400 - val_accuracy: 0.8541 - val_loss: 0.3067
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 412s 5s/step - accuracy: 0.8587 - loss: 0.3344 - val_accuracy: 0.8587 - val_loss: 0.2964
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 412s 5s/step - accuracy: 0.8760 - loss: 0.3077 - val_accuracy: 0.8848 - val_loss: 0.

In [9]:
test_loss, test_accuracy = model_resnet.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.8643 - loss: 0.2895
Test Accuracy: 87.10%


In [7]:
import cv2
import numpy as np

# Prediction function
def predict_image_resnet(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model_resnet.predict(img)[0][0]
    if prediction > 0.5:
        print("🔴 Non-Mouth Image Detected (ResNet50)")
    else:
        print("✅ Mouth Image Detected (ResNet50)")

# Example usage
predict_image_resnet("/content/drive/MyDrive/test_notMouth_img.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ Mouth Image Detected (ResNet50)
